## CW Intro to ML

This project was carried out in a school context by four classmates. 

- Pierre-Antoine ARSAGUET (pea22)
- Louis BERTHIER (ldb22)
- Nikita DMITRIEFF (nnd19)
- Ryan EL KHOURY (re122)
The subject consists in implementing a Decision Trees algorithm to indicate and classify the position of a user in one of the four rooms according to the power of his wifi.

In [110]:
#import necessary librairies

import numpy as np
from numpy.random import default_rng
import matplotlib.pyplot as plt
import pandas as pd
plt.rcParams['figure.figsize'] = [12, 6]

In [111]:
clean_dataset = np.loadtxt('wifi_db/clean_dataset.txt')
clean_dataset

array([[-64., -56., -61., ..., -82., -81.,   1.],
       [-68., -57., -61., ..., -85., -85.,   1.],
       [-63., -60., -60., ..., -85., -84.,   1.],
       ...,
       [-62., -59., -46., ..., -87., -88.,   4.],
       [-62., -58., -52., ..., -90., -85.,   4.],
       [-59., -50., -45., ..., -88., -87.,   4.]])

In [112]:
 clean_df = pd.DataFrame({'Emitter 1': clean_dataset[:, 0], 'Emitter 2': clean_dataset[:, 1], 'Emitter 3': clean_dataset[:, 2], 'Emitter 4': clean_dataset[:, 3], 'Emitter 5': clean_dataset[:, 4], 'Emitter 6': clean_dataset[:, 5], 'Emitter 7': clean_dataset[:, 6], 'Room Number': clean_dataset[:, 7]})

In [113]:
clean_df.head()

,Emitter 1,Emitter 2,Emitter 3,Emitter 4,Emitter 5,Emitter 6,Emitter 7,Room Number
0,-64.0,-56.0,-61.0,-66.0,-71.0,-82.0,-81.0,1.0
1,-68.0,-57.0,-61.0,-65.0,-71.0,-85.0,-85.0,1.0
2,-63.0,-60.0,-60.0,-67.0,-76.0,-85.0,-84.0,1.0
3,-61.0,-60.0,-68.0,-62.0,-77.0,-90.0,-80.0,1.0
4,-63.0,-65.0,-60.0,-63.0,-77.0,-81.0,-87.0,1.0


In [114]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Emitter 1    2000 non-null   float64
 1   Emitter 2    2000 non-null   float64
 2   Emitter 3    2000 non-null   float64
 3   Emitter 4    2000 non-null   float64
 4   Emitter 5    2000 non-null   float64
 5   Emitter 6    2000 non-null   float64
 6   Emitter 7    2000 non-null   float64
 7   Room Number  2000 non-null   float64
dtypes: float64(8)
memory usage: 125.1 KB


In [115]:
clean_df['Room Number'] = clean_df['Room Number'] .astype(str)
clean_df.dtypes

Emitter 1      float64
Emitter 2      float64
Emitter 3      float64
Emitter 4      float64
Emitter 5      float64
Emitter 6      float64
Emitter 7      float64
Room Number     object
dtype: object

In [116]:
clean_df.describe()

,Emitter 1,Emitter 2,Emitter 3,Emitter 4,Emitter 5,Emitter 6,Emitter 7
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,-52.330500,-55.623500,-54.964000,-53.566500,-62.640500,-80.985000,-81.726500
std,11.321677,3.417688,5.316186,11.471982,9.105093,6.516672,6.519812
min,-74.000000,-74.000000,-73.000000,-77.000000,-89.000000,-97.000000,-98.000000
25%,-61.000000,-58.000000,-58.000000,-63.000000,-69.000000,-86.000000,-87.000000
50%,-55.000000,-56.000000,-55.000000,-56.000000,-64.000000,-82.000000,-83.000000
75%,-46.000000,-53.000000,-51.000000,-46.000000,-56.000000,-77.000000,-78.000000
max,-10.000000,-45.000000,-40.000000,-11.000000,-36.000000,-61.000000,-63.000000


In [ ]:
plot = clean_df.hist(layout=(2,4), figsize=(24,12))
for ax in plot.flatten():
    ax.set_xlabel("Wifi Power")
    ax.set_ylabel("Number of observations")
plt.show()

In [ ]:
 clean_df.plot(
    title = 'Wifi power of each observation for each emitter',
    grid = True,
    xlabel = 'Observations',
    ylabel = 'Wifi Power',
)
plt.legend(loc='best')

In [ ]:
clean_df['Room Number'].value_counts()

In [29]:
# dataset read 

dataset = np.loadtxt('wifi_db/clean_dataset.txt')

print(dataset.shape)

(2000, 8)


In [30]:
dataset[:,:-1].shape


(2000, 7)

In [6]:
def read_dataset(filepath):
    """ Read in the dataset from the specified filepath

    Args:
        filepath (str): The filepath to the dataset file

    Returns:
        tuple: returns a tuple of (x, y, classes), each being a numpy array. 
               - x is a numpy array with shape (N, K), 
                   where N is the number of instances
                   K is the number of features/attributes
               - y is a numpy array with shape (N, ), and should be integers from 0 to C-1
                   where C is the number of classes 
               - classes : a numpy array with shape (C, ), which contains the 
                   unique class labels corresponding to the integers in y
    """

    x = []
    y_labels = []
    dataset = np.loadtxt(filepath)
    
    x = dataset[:,:-1]
    y = dataset[:,-1:]
    
    classes = np.unique(y)


    return (x, y, classes)


def split_dataset(x, y, test_proportion, random_generator=default_rng()):
    """ Split dataset into training and test sets, according to the given 
        test set proportion.
    
    Args:
        x (np.ndarray): Instances, numpy array with shape (N,K)
        y (np.ndarray): Class labels, numpy array with shape (N,)
        test_proprotion (float): the desired proportion of test examples 
                                 (0.0-1.0)
        random_generator (np.random.Generator): A random generator

    Returns:
        tuple: returns a tuple of (x_train, x_test, y_train, y_test) 
               - x_train (np.ndarray): Training instances shape (N_train, K)
               - x_test (np.ndarray): Test instances shape (N_test, K)
               - y_train (np.ndarray): Training labels, shape (N_train, )
               - y_test (np.ndarray): Test labels, shape (N_train, )
    """

    shuffled_indices = random_generator.permutation(len(x))
    n_test = round(len(x) * test_proportion)
    n_train = len(x) - n_test
    x_train = x[shuffled_indices[:n_train]]
    y_train = y[shuffled_indices[:n_train]]
    x_test = x[shuffled_indices[n_train:]]
    y_test = y[shuffled_indices[n_train:]]
    return (x_train, x_test, y_train, y_test)



(x, y, classes) = read_dataset('wifi_db/clean_dataset.txt')

seed = 60012
rg = default_rng(seed)
x_train, x_test, y_train, y_test = split_dataset(x, y, 
                                                 test_proportion=0.2, 
                                                 random_generator=rg)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(1600, 7) (1600, 1)
(400, 7) (400, 1)


In [7]:
np.count_nonzero(y_train == 4)

392

In [41]:
# necessary functions to compute data in order to make the decision tree



def H(y, labels): 
    
    entropy = 0
    
    
    N = x.shape[0]
    
    for label in labels:
        prob = np.count_nonzero(y == label)/N
        if prob == 0:
            continue
        entropy -= prob*np.log2(prob)
    
    return entropy

H(y_train, classes)

def remainder(y_left, y_right, labels):
    N = y_left.shape[0] + y_right.shape[0]
    
    rem = (y_left.shape[0]/N)*H(y_left, labels) + (y_right.shape[0]/N)*H(y_right, labels)
    
    return rem

def gain(y_all, y_left, y_right, labels):
    return H(y_all, labels) - remainder(y_left, y_right, labels)
            
    
    
    

In [42]:
gain(y, y_train, y_test, classes)

0.3416331631753122

In [89]:
def find_split(data, labels):
    
    n_features = len(data[0]) - 1
    
    best_gain = gain(data[:-1], data[data[:,0]>data[0][0]][:-1], data[data[:,0]<=data[0][0]][:-1], classes)
    best_thresh = data[0][0]
    best_feat = 0
    
    for i in range(n_features):
        
        best_feat_gain = gain(data[:-1], data[data[:,i]>data[i][0]][:-1], data[data[:,i]<=data[i][0]][:-1], classes)
        best_feat_thresh = data[i][0]
        
        for xi in data[:,i]:

            right = data[data[:,i]>xi]
            left = data[data[:,i]<=xi]
            
            feat_gain = gain(data[:-1], left[:-1], right[:-1], labels)
            
            if feat_gain > best_feat_gain:
                best_feat_gain = feat_gain
                best_feat_thresh = xi
                
        print(best_feat_gain, best_feat_thresh)      
        if best_feat_gain > best_gain:
            best_feat = i
            best_gain = best_feat_gain
            best_thresh = best_feat_thresh
            
            
    right = data[data[:,best_feat] > best_thresh]
    left = data[data[:,best_feat] <= best_thresh]
    
    return best_feat, best_thresh

        
        

In [87]:
best_feat, best_thresh, right, left = find_split(dataset, classes)

0.9707244042265755 -55.0
0.5094277825857527 -56.0
0.6518768781318678 -56.0
0.9123691914431387 -57.0
0.7469761444598508 -63.0
0.6955053240632996 -81.0
0.6908951549707205 -81.0


In [77]:
print(len(right), len(left))

1202 798


In [90]:
find_split(dataset, classes)

0.9707244042265755 -55.0
0.5094277825857527 -56.0
0.6518768781318678 -56.0
0.9123691914431387 -57.0
0.7469761444598508 -63.0
0.6955053240632996 -81.0
0.6908951549707205 -81.0


(0,
 -55.0,
 array([[-42., -53., -62., ..., -65., -69.,   2.],
        [-44., -55., -61., ..., -72., -68.,   2.],
        [-41., -58., -56., ..., -69., -73.,   2.],
        ...,
        [-52., -61., -48., ..., -80., -80.,   4.],
        [-54., -57., -50., ..., -83., -82.,   4.],
        [-54., -46., -48., ..., -84., -85.,   4.]]),
 array([[-64., -56., -61., ..., -82., -81.,   1.],
        [-68., -57., -61., ..., -85., -85.,   1.],
        [-63., -60., -60., ..., -85., -84.,   1.],
        ...,
        [-62., -59., -46., ..., -87., -88.,   4.],
        [-62., -58., -52., ..., -90., -85.,   4.],
        [-59., -50., -45., ..., -88., -87.,   4.]]))

In [74]:
len(data[:,0])

2000

In [91]:
len(data)

2000

In [108]:
def make_node(split_feat, split_value):
    node = {(split_feat,split_value): ({},{})}
    return node

def make_leaf(data, classes):
    N = len(data)
    result =  np.argmax([len(data[data[:,-1] == label]) for label in classes])/N
    return result



In [109]:
make_leaf(data, classes)

0.0